In this notebook I will test the rouge metric for evaluating poetry

file:///Users/jbt694/Downloads/multilingual-extension-and-evaluation-of-a-poetry-generator-.pdf

https://kavita-ganesan.com/rouge-howto/

https://en.wikipedia.org/wiki/ROUGE_(metric)

In [1]:
from rouge_metric import PyRouge

import pandas as pd
import numpy as np

In [2]:
hypotheses = ["A big green desk"]
#hypotheses = ["I was having a beer"]


references = ["A big red desk"]

# Evaluate document-wise ROUGE scores
rouge = PyRouge(rouge_n=(1, 2, 3, 4),
                rouge_l=True,
                rouge_w=False,
                #rouge_w_weight=1.2,
                rouge_s=False,
                rouge_su=True,
                 skip_gap=4
               )
scores = rouge.evaluate(hypotheses, [references])
scores

{'rouge-1': {'r': 0.75, 'p': 0.75, 'f': 0.75},
 'rouge-2': {'r': 0.3333333333333333,
  'p': 0.3333333333333333,
  'f': 0.3333333333333333},
 'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
 'rouge-l': {'r': 0.75, 'p': 0.75, 'f': 0.75},
 'rouge-su4': {'r': 0.5555555555555556,
  'p': 0.5555555555555556,
  'f': 0.5555555555555556}}

In [3]:
rouge = PyRouge(rouge_n=(1, 2, 3, 4),
                rouge_l=True,
                rouge_w=False,
                rouge_s=False,
                rouge_su=True,
                 skip_gap=4
               )


### Compare poem1 with poem2 line by line

In [4]:
f=open("data/poem1.txt")
lines1 = [line.strip() for line in f if len(line.strip())>0]
f.close()

f=open("data/poem2.txt")
lines2 = [line.strip() for line in f if len(line.strip())>0]
f.close()


results = []
for indline, (line1,line2) in enumerate(zip(lines1,lines2)):
    scores = rouge.evaluate([line1], [[line2]])
    results.append(scores)
    

In [5]:
accumulated_results=pd.DataFrame(results[0])
for results_line in results[1:]:
    accumulated_results += pd.DataFrame(results_line)

In [6]:
accumulated_results

#This is so impressive. There is no overlap at all between these two poems.

,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rouge-su4
r,0.0,0.0,0.0,0.0,0.0,0.0
p,0.0,0.0,0.0,0.0,0.0,0.0
f,0.0,0.0,0.0,0.0,0.0,0.0


Let's now make a test evaluating a poem that has been created based on another one.

In [7]:
#poem2-slightlyplagiarized.txt

f=open("data/poem2.txt")
lines1 = [line.strip() for line in f if len(line.strip())>0]
f.close()

f=open("data/poem2-slightlyplagiarized.txt")
lines2 = [line.strip() for line in f if len(line.strip())>0]
f.close()

results = []
for indline, (line1,line2) in enumerate(zip(lines1,lines2)):
    scores = rouge.evaluate([line1], [[line2]])
    results.append(scores)

accumulated_results=pd.DataFrame(results[0])
for results_line in results[1:]:
    accumulated_results += pd.DataFrame(results_line)
    
accumulated_results

,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rouge-su4
r,11.590476,8.35,5.15,4.25,11.590476,9.946642
p,11.590476,8.35,5.15,4.25,11.590476,9.946642
f,11.590476,8.35,5.15,4.25,11.590476,9.946642


We have to normalize the results with the number of lines

In [8]:
accumulated_results/len(results)

,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rouge-su4
r,0.827891,0.596429,0.367857,0.303571,0.827891,0.710474
p,0.827891,0.596429,0.367857,0.303571,0.827891,0.710474
f,0.827891,0.596429,0.367857,0.303571,0.827891,0.710474


Yes... we can see that there is a bit of plagiarization.

### Compare poem1 and poem2
#### line1 with line1,2,3,4...
#### line2 with line1,2,3,4...

In [9]:
#poem2-slightlyplagiarized.txt

f=open("data/poem2.txt")
lines1 = [line.strip() for line in f if len(line.strip())>0]
f.close()

f=open("data/poem2-slightlyplagiarized.txt")
lines2 = [line.strip() for line in f if len(line.strip())>0]
f.close()

results = []
for indline1, line1 in enumerate(lines1):
    for indline2,line2 in enumerate(lines2):
        scores = rouge.evaluate([line1], [[line2]])
        results.append(scores)

accumulated_results=pd.DataFrame(results[0])
for results_line in results[1:]:
    accumulated_results += pd.DataFrame(results_line)
    
accumulated_results/len(results)

,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rouge-su4
r,0.150267,0.055102,0.026276,0.021684,0.150267,0.094453
p,0.150449,0.055102,0.026276,0.021684,0.150449,0.094335
f,0.149673,0.055074,0.026276,0.021684,0.149673,0.093475


### Compare poem1 and poem2 without considering the lines

#### I will concatenate all the lines together with " "

In [19]:
#poem2-slightlyplagiarized.txt

f=open("data/poem2.txt")
poem1 = " ".join([line.strip() for line in f if len(line.strip())>0])
f.close()

f=open("data/poem2-slightlyplagiarized.txt")
poem2 = " ".join([line.strip() for line in f if len(line.strip())>0])
f.close()

scores = rouge.evaluate([poem1], [[poem2]])

pd.DataFrame(scores)


,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rouge-su4
r,0.84058,0.676471,0.522388,0.378788,0.84058,0.721106
p,0.84058,0.676471,0.522388,0.378788,0.84058,0.721106
f,0.84058,0.676471,0.522388,0.378788,0.84058,0.721106


### Check one poem for its intra novelty

Now let's check the intra novelty.

In [11]:
f=open("data/poem2.txt")
lines = [line.strip() for line in f if len(line.strip())>0]
f.close()

results = []
for indline1, line1 in enumerate(lines):
    for indline2, line2 in enumerate(lines):
        if indline1 != indline2:
            scores = rouge.evaluate([line1], [[line2]])
            results.append(((indline1,indline2),scores))

In [12]:
results

[((0, 1),
  {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-su4': {'r': 0.0, 'p': 0.0, 'f': 0.0}}),
 ((0, 2),
  {'rouge-1': {'r': 0.2, 'p': 0.2, 'f': 0.20000000000000004},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r': 0.2, 'p': 0.2, 'f': 0.20000000000000004},
   'rouge-su4': {'r': 0.07142857142857142,
    'p': 0.07142857142857142,
    'f': 0.07142857142857142}}),
 ((0, 3),
  {'rouge-1': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-2': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-3': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-4': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-l': {'r': 0.0, 'p': 0.0, 'f': 0.0},
   'rouge-su4': {'r': 0.0, 'p': 0.0, 'f': 0.0}}),
 ((0, 4),
  {'rouge-1': {'r': 0.0, 'p': 0.

In [13]:
accumulated_results=pd.DataFrame(results[0][1])
for results_line in results[1:]:
    accumulated_results += pd.DataFrame(results_line[1])
    
accumulated_results

,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rouge-su4
r,21.547619,6.450000,6.0,6.0,21.547619,11.897070
p,21.547619,6.450000,6.0,6.0,21.547619,11.897070
f,21.392152,6.444444,6.0,6.0,21.392152,11.703289


In [20]:
accumulated_results/len(results)

,rouge-1,rouge-2,rouge-3,rouge-4,rouge-l,rouge-su4
r,0.118394,0.035440,0.032967,0.032967,0.118394,0.065369
p,0.118394,0.035440,0.032967,0.032967,0.118394,0.065369
f,0.117539,0.035409,0.032967,0.032967,0.117539,0.064304


In [21]:
!cat data/poem2.txt

en afección y achaque partidas
bandos afectas gustos estragados
de cálidos amores y cuidados
bandos afectas gustos estragados

cartas calenturas sobres amores
dulces en el cariño y sufridores
afección afectuosa negro pelo
en su amor cada uno dellos cielo

recios miembros de amores bien dulces
de cariños afectas gallardetes
bandos afectas gustos estragados
en el tierno amor ejercitados

saldré de la emoción y amor
afección de sensaciones y dalias